# Cross Lingual summarization CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time
from rouge_score import rouge_scorer
from rouge_score import scoring

In [2]:
if not tf.config.list_physical_devices('GPU'):
    print("Change runtime to \"GPU runtime\" for faster computations")

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Params

In [4]:
BATCH_SIZE = 8

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [5]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275008 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


In [7]:
ckpt_file = "../models/checkpoint_t5_en_de_cross_lingual_plus_translation.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [8]:
en_de_prefix = tf.reshape(tokenizer.encode("summarize: en_to_ger ", return_tensors="tf"), (-1,))
de_en_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_en ", return_tensors="tf"), (-1,))
en_en_prefix = tf.reshape(tokenizer.encode("summarize: en_to_en ", return_tensors="tf"), (-1,))
de_de_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_ger ", return_tensors="tf"), (-1,))

In [9]:
prefix_length = de_de_prefix.shape[0]
prefix_length

9

In [10]:
import numpy as np
MAX_ARTICLE_LEN = 512
MAX_HIGHLIGHT_LEN = 150
GLOBAL_BATCH_SIZE = 8

def get_tfrecord_dataset(file_name):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
    }

    dataset = tf.data.TFRecordDataset(f"../data/{file_name}.tfrecord")

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        return [
            tf.strided_slice(tf.concat([de_de_prefix, record['ger_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.strided_slice(tf.concat([en_de_prefix, record['en_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.strided_slice(tf.concat([de_en_prefix, record['ger_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
            tf.strided_slice(tf.concat([en_en_prefix, record['en_x']], axis=0), [0], [-1]), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
        ]
    
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(100)
    return dataset.batch(GLOBAL_BATCH_SIZE)

test_ds = get_tfrecord_dataset("corss_lingual_test_cnn_daily_mail")

In [11]:
def get_summaries(ds):
    for i in range(1,5):
        yield ds[(i-1)*4], ds[i*4-3], ds[i*4-2], ds[i*4-1]


for ds in test_ds.take(1):
    for i in get_summaries(ds):
        print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)

(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)


## Evaluation
### Define Rouge Score

In [12]:
class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Summaries

In [31]:
predictions = []
start_time = time.time()

for i, ds_item in enumerate(test_ds): 
    for (input_ids, input_mask, y, y_ids) in get_summaries(ds_item):
        summaries = model.generate(
            input_ids=input_ids, 
            attention_mask=input_mask, 
            early_stopping=True, 
            max_length=150
        )

        articles = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]

        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
        real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
        for pred_sent, real_sent, article_sent in zip(pred, real, articles):
            predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 20:
#         otherwise it will take ages
        break


# rouge_score.result()

0 : time genreate batch: 18.250395369529723
10 : time genreate batch: 181.13052678108215
20 : time genreate batch: 162.7270177602768


### Lets have a look at some of these predicted summaries

In [32]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [33]:
print(get_random_prediction())

article: summarize: en_to_ger Arsenals midfield trio Jack Wilshere, Mikel Arteta and Abou Diaby have all been handed starts for the club's Under 21s game on Tuesday night as they continue their respective recoveries from injury. Steve Gatting's young Gunners take on Stoke City at the Emirates Stadium in a U21 Premier League clash. Teenage winger Serge Gnabry is also in the starting line-up, alongside a host of promising youngsters including Dan Crowley. Jack Wilshere, pictured in first team training last week, starts for Arsenal Under 21s on Tuesday night . Club captain Mikel Arteta (left) is also in the team to play against Stoke City at the Emirates Stadium . Wilshere watched Arsenal beat Liverpool 4-1 in the Premier League from the stands on Saturday afternoon . Iliev, Maitland-Niles, O’Connor, Bielik, Ormonde-Ottewill, Diaby, Arteta, Gnabry, Wilshere, Crowley, Iwobi . Wilshere has been out of action since November after breaking his foot against Manchester United, as has club capta

In [34]:
print(get_random_prediction())

article: summarize: ger_to_ger (CNN) Warren Weinstein, der anscheinend der einzige amerikanische Staatsbürger war, der von al-Qaida als Geisel gehalten wurde, wurde im Januar bei einem US-Drohnenangriff versehentlich getötet. Aber es musste nicht so sein. Ein hochrangiger US-Regierungsvertreter, der mit dem Umgang mit dem Thema vertraut ist, sagte CNN, die US-Regierung habe keine ernsthaften Bemühungen unternommen, über die Freilassung des 73-jährigen Entwicklungsexperten zu verhandeln, weder direkt an al-Qaida noch über Stellvertreter in Pakistan. Ein weiterer hochrangiger US-Regierungsvertreter sagte CNN, Weinsteins Gefangennahme durch Al Kaida habe es den Vereinigten Staaten schwer gemacht zu verhandeln, obwohl Stellvertreter wie die pakistanische Regierung Verbindungen zu Mittelsmännern hätten, die möglicherweise geholfen hätten. Ein hochrangiger pakistanischer Beamter sagte CNN, dass die pakistanische Regierung nach der Entführung Weinsteins ihre Fühler nach Mitgliedern des milita

In [35]:
print(get_random_prediction())

article: summarize: ger_to_ger Der Labour-Abgeordnete John Prescott hat Prinz Charles gegen Vorwürfe verteidigt, er versuche, die Regierungspolitik heimlich mit gekritzelten privaten Notizen an Minister zu beeinflussen - und beharrt darauf, er solle "das Recht haben, so viele verdammte Briefe zu schreiben, wie er will". Der ehemalige Vizepremier sagte, er sehe kein Problem darin, dass der künftige König an die Minister der Regierung schreibe und beharrte darauf, dass er "diesem Land viel zu bieten" habe. Lord Prescotts Intervention erfolgte, nachdem der Oberste Gerichtshof ein früheres Urteil unterstützt hatte, das den Weg für die Veröffentlichung von Prinz Charles "so genannten" Black Spider "-Memos ebnete. Lord Prescott, links, sagte, dass der Prinz von Wales dem Land viel zu bieten habe. Die Briefe wurden zwischen September 2004 und März 2005 verfasst und vom Prinzen an sieben Regierungsministerien geschickt. Lord Prescott, dessen Briefe getrennt von denen veröffentlicht werden soll

In [36]:
print(get_random_prediction())

article: summarize: ger_to_ger Ian Murray rechnet damit, dass ein vermeintlich leichterer Anlauf im letzten Monat der Meisterschaftssaison den zweiten Platz vor den Rangers sichern kann. Seine Dumbarton-Mannschaft hat es in dieser Legislaturperiode mit seinen beiden ehemaligen Teams in der Liga zu tun bekommen und wird dies in den letzten Wochen des Wahlkampfs noch einmal tun. Doch während Stuart McCalls Männer derzeit das Momentum haben - Hibs und Cowdenbeath schlugen, während Alan Stubbs Männer ebenfalls gegen Raith verloren - wäre Murray nicht überrascht, wenn sich das Kräfteverhältnis bald wieder in Richtung Osterstraße verschieben würde. Dumbarton-Chef Ian Murray glaubt, dass die Rangers sich schwer tun werden, ihr Tempo im Vorfeld aufrecht zu erhalten. Die Rangers - derzeit punktgleich, aber mit einem Spiel weniger - haben noch zwei Spiele gegen Titelgewinner Hearts zu bestreiten, darunter das Spiel am Sonntag in Ibrox, während die Leith-Mannschaft es einmal mehr mit ihren Stadtr

In [37]:
print(get_random_prediction())

article: summarize: en_to_en England captain Alastair Cook completed a much-needed century on the second morning of Englands opening tour match in the West Indies. Cook resumed on 95 and reached three figures with minimal fuss before retiring out. England captain Alastair Cook completed a century on the second morning of Englands opening tour match . A controlled thick edge from the first ball of the day brought him an 11th boundary of the innings and in the following over he punched the ball for two off the back foot. He offered a gentle wave of the bat and a handshake to partner Gary Ballance and walked off on 101 from 200 deliveries. That allowed Ian Bell to arrive at the crease, with batting time more important to the tourists than attempting to force a result in this two-day fixture. Ian Bell plays to the offside during day two of the St Kitts and Nevis Invitational XI versus England tour match .

pred sentence: England-Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffn

In [38]:
for i in range(10):
    print(get_random_prediction())

article: summarize: ger_to_ger Wissenschaftler glauben, dass sich das Leben auf der Erde vor etwa 3,8 Milliarden Jahren entwickelt hat. Doch obwohl sie in der Lage waren, ein Datum für das Erscheinen des Lebens festzulegen, wissen sie noch lange nicht, wie es aussah. Nun sagen Forscher in den USA und Italien, sie hätten Beweise dafür, dass DNA-ähnliche Fragmente vor vier Milliarden Jahren mit "Anleitungen" versehen gewesen sein könnten, die ihr Wachstum in komplexe Lebensformen gelenkt hätten. Scrollen Sie nach unten für ein Video: Wissenschaftler glauben, dass sich das Leben auf der Erde vor etwa 3,8 Milliarden Jahren entwickelt hat, aber sie wissen noch lange nicht, wie es entstanden ist. Nun sagen Forscher in den USA und Italien, sie hätten Beweise dafür, dass DNA-ähnliche Fragmente vor 4 Milliarden Jahren mit "Anleitungen" geliefert worden sein könnten, die ihr Wachstum in komplexe Lebensformen gelenkt haben. RNA, die für Ribonukleinsäure steht, ist ein Molekül, das aus einem oder 

## Save results to text file

In [39]:
result_path = "../results/result_02.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")

## Load save File

In [40]:
data_points = []
result_path = "../results/result_02.txt"
file = open(result_path, "r")
for line in file:
    data_points.append(line)

In [42]:
data_points[4]

'real sentence: Justizabteilung In einem 7-seitigen Brief an den Sprecher des Repräsentantenhauses, John Boehner, erklärte Lerner, warum der ehemalige IRS-Beamte sich auf den fünften Zusatzartikel berufen durfte. Der mit dem Fall betraute Bundesstaatsanwalt schickte seine Entscheidung am letzten Tag vor Inkrafttreten seines eigenen Rücktritts an das Capitol Hill. Lerner gab bei einer Anhörung 2013 eine eigennützige Eröffnungserklärung ab, weigerte sich aber, Fragen zu stellen, obwohl sie unter Vorladung stand. Die Anhörung konzentrierte sich auf die Gewohnheit\n'

In [43]:
class SummaryData():
    
    def __init__(self):
        self.language_tag = ''
        self.real_data = ''
        self.pred_data = ''        

In [44]:
count = 0
summary_data = SummaryData()
summary_data_list = []
for point in data_points:
    count += 1
    
    if count == 1:
        summary_data.language_tag = point.split(" ")[2]
    elif count == 3:
        summary_data.pred_data = ": ".join(point.split(":")[1:])
    elif count == 5:
        summary_data.real_data = ": ".join(point.split(":")[1:])
        summary_data_list.append(summary_data)
        summary_data = SummaryData()
        count = 0
summary_data_list[0].__dict__

{'language_tag': 'ger_to_ger',
 'real_data': ' Justizabteilung In einem 7-seitigen Brief an den Sprecher des Repräsentantenhauses, John Boehner, erklärte Lerner, warum der ehemalige IRS-Beamte sich auf den fünften Zusatzartikel berufen durfte. Der mit dem Fall betraute Bundesstaatsanwalt schickte seine Entscheidung am letzten Tag vor Inkrafttreten seines eigenen Rücktritts an das Capitol Hill. Lerner gab bei einer Anhörung 2013 eine eigennützige Eröffnungserklärung ab, weigerte sich aber, Fragen zu stellen, obwohl sie unter Vorladung stand. Die Anhörung konzentrierte sich auf die Gewohnheit\n',
 'pred_data': ' Der Sprecher des Repräsentantenhauses, John Boehner, teilte mit, dass Lerner sich in den fünften Zusatzartikeln der US-Verfassung einwickeln könne. Lerner steht seit langem im Fadenkreuz der GOP, weil sie die IRS-Abteilung leitete, die mit der Genehmigung von Anträgen gemeinnütziger Gruppen auf steuerfreien Status betraut ist.\n'}

In [45]:
from ast import literal_eval

results_en_trans = []
results_en_path = "../results/en_en_results"
file = open(results_en_path, "r")
for i, line in enumerate(file):
    results_en_trans.append(literal_eval(line))
    
results_ger_trans = []
results_ger_path = "../results/en_en_results"
file = open(results_ger_path, "r")
for i, line in enumerate(file):
    results_ger_trans.append(literal_eval(line))

In [46]:
rouge_scores_dict = dict()
rouge_scores_dict['en_to_en'] = RougeScore()
# rouge_scores_dict['en_to_en_trans'] = RougeScore()

rouge_scores_dict['en_to_ger'] = RougeScore()
rouge_scores_dict['ger_to_en'] = RougeScore()
rouge_scores_dict['ger_to_ger'] = RougeScore()
# rouge_scores_dict['ger_to_ger_trans'] = RougeScore()

rouge_scores_dict

{'en_to_en': <__main__.RougeScore at 0x7f358c4bde80>,
 'en_to_ger': <__main__.RougeScore at 0x7f358c4bd0f0>,
 'ger_to_en': <__main__.RougeScore at 0x7f358c179f28>,
 'ger_to_ger': <__main__.RougeScore at 0x7f3574791f28>}

In [47]:
for summary_data in summary_data_list:
    rouge_scores_dict[summary_data.language_tag](summary_data.real_data, summary_data.pred_data)
    

In [48]:
for key, rouge_score_item in rouge_scores_dict.items():
    print(key, rouge_score_item.result())
    print()

rouge1 = 11.38, 95% confidence [10.22, 12.38]
rouge2 = 3.93, 95% confidence [3.35, 4.59]
rougeLsum = 10.62, 95% confidence [9.66, 11.63]
en_to_en {'rouge1': 11.377223619372618, 'rouge2': 3.926394264914629, 'rougeLsum': 10.622083612293935}

rouge1 = 33.86, 95% confidence [32.31, 35.61]
rouge2 = 13.73, 95% confidence [12.44, 15.06]
rougeLsum = 23.92, 95% confidence [22.60, 25.24]
en_to_ger {'rouge1': 33.85660986812824, 'rouge2': 13.73331925809661, 'rougeLsum': 23.919439511259714}

rouge1 = 9.74, 95% confidence [8.84, 10.80]
rouge2 = 3.29, 95% confidence [2.75, 3.93]
rougeLsum = 9.15, 95% confidence [8.28, 10.19]
ger_to_en {'rouge1': 9.741955767604104, 'rouge2': 3.2872073062236495, 'rougeLsum': 9.146500119759168}

rouge1 = 31.24, 95% confidence [29.70, 32.88]
rouge2 = 12.03, 95% confidence [10.77, 13.52]
rougeLsum = 21.48, 95% confidence [20.07, 22.82]
ger_to_ger {'rouge1': 31.24288565519981, 'rouge2': 12.032575111047011, 'rougeLsum': 21.48437773872943}



# Special Example

In [29]:
input_text = "In a world where we have to read and understand a lot of documents automatic text summarization has an obvious demand. To have the option to get a brief summary of a text in your language can be very useful. But would it not be even more useful, if you could have the option to get a summary of a text in a language, you do not understand, in your wished language? Sometimes we don’t want an exact translation, sometimes we just want to know a brief overview of a text in a language we don’t understand. That is the case where Cross-Lingual summarization would be prefered over normal translation. Cross-Lingual summarization can give you a short overview of a text in a language you do not understand."

In [30]:
input_text_test = "The dramatic growth of data on the internet leads to the need to automatically process and understand the data. A big part of the data is text data in many languages. This overwhelming amount of information causes a demand for automatic text summarization and other Natural Language Processing (NLP) taks.\n The Field of NLP had some high points in the last couple of years, when the field got revolutionized by Neural Language Models. With publications like Attention is all you need \cite{vaswani2017attention} or GPT3\cite{brown2020language} the limits of the field are pushed even further. Because of large data corpuses, scraped from the internet, and advanced models which contain up to 175 Billion parameters like the GPT3 it is possible to generate text, answer questions, summarize text, translate or many other things.\n In this master thesis we will take a deeper look into summarization. There are extractive and abstractive techniques to summarization. The extractive summary technique tries to find subsets of sentences, which representante the original text well and uses them to summarize the original text \cite{allahyari2017text}. The abstractive technique uses advanced language models to generate a new text, which should be much shorter than the original one, contain all the key information and preserve the overall meaning.\n In this master thesis we will use the abstractive technique, because it is closer to a human-like interpretation. It combines the ability to understand what the context of a given text is and the ability to generate fluent and grammatically correct text to that given context.\n The summarization in one language is an interesting topic but cross lingual approach is even more interesting. The goal of Cross Lingual Summarization is to summarize a text from one language into another language. It combines the ability to summarize and the ability to translate. Where state of the art models perform well on normal summarization, it will be interesting to see how well they perform doing Cross-Lingual Summarization."

In [31]:
x = tokenizer.encode_plus("summarize: en_to_ger " + input_text, max_length=512, return_tensors="tf", padding='max_length', truncation=True)
input_ids = tf.reshape(x['input_ids'], (1,-1))
attention_mask = tf.reshape(x['attention_mask'], (1,-1))

In [32]:
print(input_ids.shape, attention_mask.shape)

(1, 512) (1, 512)


In [33]:
summaries = model.generate( 
    input_ids=input_ids, 
    attention_mask=attention_mask
)

In [34]:
summaries

<tf.Tensor: shape=(1, 74), dtype=int32, numpy=
array([[    0,    86,   645,  3779,     6,    16,    74,   558,  2584,
            3, 20127,    15,   110,    35,    64, 19163,  3766,     6,
          229, 15820,    15,  5027,     7,    63,    29, 19712,   266,
        30410,    15,  5222,  6367,     5,  4098,  6199,     3,    15,
            7,   311, 30001,    49,     6,  1301,   292,    67,  7251,
         8219,    29,     6,   266, 11068, 14449,   266,     7,  5027,
           15,     7,    16,   645, 16933,   170,  8837,     6,    67,
          292,   311, 19163,     6,    16,  1197,    52, 24054,    29,
        16933,    58]], dtype=int32)>

In [35]:
tokenizer.decode(summaries[0])

'<pad> In einer Welt, in der wir viele Dokumente lesen und verstehen müssen, ist automatische Textsynthese eine offensichtliche Forderung. Aber wäre es nicht nützlicher, wenn Sie die Möglichkeit hätten, eine Zusammenfassung eines Textes in einer Sprache zu bekommen, die Sie nicht verstehen, in Ihrer gewünschten Sprache?'